# ingest.stores.factory

> Convenience class to create Vector Stores.

In [ ]:
# | default_exp ingest.stores.factory

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

from onprem.ingest.stores.dense import DenseStore, ChromaStore
from onprem.ingest.stores.sparse import SparseStore, WhooshStore, ElasticsearchSparseStore, SharePointStore
from onprem.ingest.stores.dual import DualStore, ElasticsearchStore

class VectorStoreFactory:
    """
    Convenience factory for creating vector stores with sensible defaults.
    
    Provides a simple interface to create the most commonly used vector stores:
    - ChromaStore (default) - Dense vector store using Chroma
    - WhooshStore - Sparse text search using Whoosh
    - ElasticsearchStore - Unified dense + sparse using Elasticsearch
    """
    
    @classmethod
    def create(cls, 
               kind: str = 'chroma',
               persist_location: str = None,
               **kwargs):
        """
        Create a vector store instance.
        
        Args:
            kind: Type of store to create. One of:
                  - 'chroma' (default): ChromaStore for dense vector search
                  - 'whoosh': WhooshStore for sparse text search  
                  - 'chroma+-whoosh': a DualStore using  ChromaStore and WhooshStore 
                  - 'elasticsearch': ElasticsearchStore for unified dense + sparse
                  - 'elasticsearch_sparse': For use with pre-existing Elasticsearch indices without dense vectors
            persist_location: Where to store the index/database
            **kwargs: Additional arguments passed to the store constructor
            
        Returns:
            VectorStore instance
            
        Examples:
            # Create default ChromaStore
            store = VectorStoreFactory.create()
            
            # Create WhooshStore for text search
            store = VectorStoreFactory.create('whoosh', persist_location='./search_index')
            
            # Create ElasticsearchStore for hybrid search
            store = VectorStoreFactory.create('elasticsearch', 
                                             persist_location='http://localhost:9200',
                                             index_name='my_docs')
        """
        if kind == 'chroma':
            return ChromaStore(persist_location=persist_location, **kwargs)
        elif kind == 'whoosh':
            return WhooshStore(persist_location=persist_location, **kwargs)
        elif kind== 'chroma+whoosh':
            # Let DualStore handle None persist_location with its own defaults
            if persist_location is not None:
                dense_path = os.path.join(persist_location, 'dense')
                sparse_path = os.path.join(persist_location, 'sparse')
            else:
                dense_path = None
                sparse_path = None
            return DualStore(
                    sparse_persist_location=sparse_path,
                    dense_persist_location=dense_path,
                    **kwargs
                )
        elif kind == 'elasticsearch':
            return ElasticsearchStore(persist_location=persist_location, **kwargs)
        elif kind == 'elasticsearch_sparse':
            return ElasticsearchSparseStore(
                        persist_location=persist_location, # 'https://localhost:9200'
                        **kwargs                           # e.g., index_name, basic_auth, etc.
                    )
        elif kind=='sharepoint':
            return SharePointStore(persist_location=persist_location, # https://sharepoint.your-company.com
                                   **kwargs)
        else:
            raise ValueError(f"Unknown store kind: {kind}. Supported vector store types: 'chroma', 'whoosh', 'chroma-whoosh', 'elasticsearch', 'elasticsearch-sparse")


In [ ]:
show_doc(VectorStoreFactory.create)

---

[source](https://github.com/amaiya/onprem/blob/master/onprem/ingest/stores/factory.py#L24){target="_blank" style="float:right; font-size:smaller"}

### VectorStoreFactory.create

>      VectorStoreFactory.create (kind:str='chroma', persist_location:str=None,
>                                 **kwargs)

*Create a vector store instance.

Args:
    kind: Type of store to create. One of:
          - 'chroma' (default): ChromaStore for dense vector search
          - 'whoosh': WhooshStore for sparse text search  
          - 'elasticsearch': ElasticsearchStore for unified dense + sparse
    persist_location: Where to store the index/database
    **kwargs: Additional arguments passed to the store constructor

Returns:
    VectorStore instance

Examples:
    # Create default ChromaStore
    store = VectorStoreFactory.create()

    # Create WhooshStore for text search
    store = VectorStoreFactory.create('whoosh', persist_location='./search_index')

    # Create ElasticsearchStore for hybrid search
    store = VectorStoreFactory.create('elasticsearch', 
                                     persist_location='http://localhost:9200',
                                     index_name='my_docs')*

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()